# Lattice Dynamics under DissDTWA

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import matplotlib
from numpy import cos, sin
from numpy import sqrt
import time
from numba import jit
from numpy import sin, cos

"""In order to provide a better presentation of the graphs we use the rcParams options shown below."""

matplotlib.rcParams['text.usetex'] = False
matplotlib.rcParams['mathtext.fontset'] = 'stix'
matplotlib.rcParams['font.family'] = 'STIXGeneral'
matplotlib.rcParams['font.size'] = 15
matplotlib.rcParams['figure.figsize'] = (11.0, 8.0)


label_size = 20


In [2]:

import matplotlib.pyplot as plt
%matplotlib inline
from numpy import random, zeros, sqrt,cos,sin

import matplotlib.pyplot as plt
import numpy as np
import sdeint
import time
from sdeint import itoint, stratint, itoSRI2, stratKP2iS,itoEuler
from numba import jit
import numbers

from numpy import sqrt, zeros, diag, random, real, array, complex_, transpose, conjugate, exp, matmul, real


import matplotlib
import matplotlib.pyplot as plt
from scipy.linalg import norm
from numpy import dot

from numpy import pi as pi
import numba
from numpy import array

import scipy

from numpy import exp, imag, real,zeros, conj
from numpy import sum
from scipy import sparse
from numpy import array
import scipy.sparse as sp
from numpy import concatenate

from scipy.sparse import csr_matrix
from numpy import sqrt, dot
from numpy.random import choice
from numpy.random import normal
from numpy import sqrt


from numpy import add
from numba import jit

In [3]:
%reload_ext Cython

The set of equations to solve are:


\begin{align*}
{\rm d}\left\langle \sigma_{x}^{k}\right\rangle  & =\left(-\Delta\left\langle \sigma_{y}^{k}\right\rangle +2C_{0}^{0}\Omega\left\langle \sigma_{z}^{k}\right\rangle \mathcal{I}\left({\rm e}^{{\rm i}\overrightarrow{{\rm k}}_{{\rm L}}\cdot\overrightarrow{r}_{k}}\left(\hat{e}_{{\rm L}}\cdot\hat{e}_{0}^{*}\right)\right)-\left(C_{0}^{0}\right)^{2}\left(\overrightarrow{\mathcal{R}}_{0,0}^{k}\cdot\left\langle \overrightarrow{\sigma}_{y}\right\rangle \right)\left\langle \sigma_{z}^{k}\right\rangle \right.\\
 & \,\,\,\,\,\left.+\left(C_{0}^{0}\right)^{2}\Gamma\left(\left\langle S^{x}\right\rangle \left\langle \sigma_{z}^{k}\right\rangle -\frac{\left\langle \sigma_{x}^{k}\right\rangle \left(1+\left\langle \sigma_{z}^{k}\right\rangle \right)}{2}\right)\right){\rm d}t-C_{0}^{0}\sqrt{\Gamma}\left\langle \sigma_{y}^{k}\right\rangle {\rm d}W,\\
{\rm d}_{t}\left\langle \sigma_{y}^{k}\right\rangle  & =\left(\Delta\left\langle \sigma_{x}^{k}\right\rangle +2C_{0}^{0}\Omega\left\langle \sigma_{z}^{k}\right\rangle \mathcal{R}\left({\rm e}^{{\rm i}\overrightarrow{{\rm k}}_{{\rm L}}\cdot\overrightarrow{r}_{k}}\left(\hat{e}_{{\rm L}}\cdot\hat{e}_{0}^{*}\right)\right)+\left(C_{0}^{0}\right)^{2}\left\langle \sigma_{z}^{k}\right\rangle \left(\overrightarrow{\mathcal{R}}_{0,0}^{k}\cdot\left\langle \overrightarrow{\sigma}_{x}\right\rangle \right)\right.\\
 & \,\,\,\,\,\left.+\left(C_{0}^{0}\right)^{2}\Gamma\left(\left\langle S^{y}\right\rangle \left\langle \sigma_{z}^{k}\right\rangle -\frac{\left\langle \sigma_{y}^{k}\right\rangle \left(1+\left\langle \sigma_{z}^{k}\right\rangle \right)}{2}\right)\right){\rm d}t+C_{0}^{0}\sqrt{\Gamma}\left\langle \sigma_{x}^{k}\right\rangle {\rm d}W,\\
{\rm d}_{t}\left\langle \sigma_{z}^{k}\right\rangle  & =\left(-2C_{0}^{0}\Omega\left(\left\langle \sigma_{x}^{k}\right\rangle \mathcal{I}\left({\rm e}^{{\rm i}\overrightarrow{{\rm k}}_{{\rm L}}\cdot\overrightarrow{r}_{k}}\left(\hat{e}_{{\rm L}}\cdot\hat{e}_{0}^{*}\right)\right)+\left\langle \sigma_{y}^{k}\right\rangle \mathcal{R}\left({\rm e}^{{\rm i}\overrightarrow{{\rm k}}_{{\rm L}}\cdot\overrightarrow{r}_{k}}\left(\hat{e}_{{\rm L}}\cdot\hat{e}_{0}^{*}\right)\right)\right)\right.\\
 & \,\,\,\,\,\left.-\left(C_{0}^{0}\right)^{2}\overrightarrow{\mathcal{R}}_{0,0}^{k}\cdot\left(\left\langle \overrightarrow{\sigma}_{x}\right\rangle \left\langle \sigma_{y}^{k}\right\rangle -\left\langle \sigma_{x}^{k}\right\rangle \left\langle \overrightarrow{\sigma}_{y}\right\rangle \right)-\left(C_{0}^{0}\right)^{2}\Gamma\left(\left\langle \sigma_{x}^{k}\right\rangle \left\langle S^{x}\right\rangle +\left\langle \sigma_{y}^{k}\right\rangle \left\langle S^{y}\right\rangle +1+\left\langle \sigma_{z}^{k}\right\rangle -\frac{\left\langle \sigma_{x}^{k}\right\rangle ^{2}+\left\langle \sigma_{y}^{k}\right\rangle ^{2}}{2}\right)\right){\rm d}t\\
 & \,\,\,\,\,+C_{0}^{0}\sqrt{\Gamma}\left(1+\left\langle \sigma_{z}^{k}\right\rangle \right){\rm d}W.
\end{align*}

Writing in matricial form:


\begin{align*}
\left\langle \overrightarrow{s}_{k}\right\rangle  & =\begin{pmatrix}\left\langle \sigma_{x}^{k}\right\rangle \\
\left\langle \sigma_{y}^{k}\right\rangle \\
\left\langle \sigma_{z}^{k}\right\rangle 
\end{pmatrix},\\
{\rm d}\left\langle \overrightarrow{s}_{k}\right\rangle  & =A_{k}\left\langle \overrightarrow{s}_{k}\right\rangle {\rm d}t-\left(C_{0}^{0}\right)^{2}\Gamma\begin{pmatrix}0\\
0\\
1
\end{pmatrix}{\rm d}t+C_{0}^{0}\sqrt{\Gamma}\begin{pmatrix}-\left\langle \sigma_{y}^{k}\right\rangle \\
 \left\langle \sigma_{x}^{k}\right\rangle \\
 1+\left\langle \sigma_{z}^{k}\right\rangle 
\end{pmatrix}{\rm d}W\\
A_{k}^{xx} & =-\left(C_{0}^{0}\right)^{2}\Gamma\frac{\left(1+\left\langle \sigma_{z}^{k}\right\rangle \right)}{2},\\
A_{k}^{xy} & =-\Delta,\\
A_{k}^{xz} & =2C_{0}^{0}\Omega\mathcal{I}\left({\rm e}^{{\rm i}\overrightarrow{{\rm k}}_{{\rm L}}\cdot\overrightarrow{r}_{k}}\left(\hat{e}_{{\rm L}}\cdot\hat{e}_{0}^{*}\right)\right)-\left(C_{0}^{0}\right)^{2}\left(\overrightarrow{\mathcal{R}}_{0,0}^{k}\cdot\left\langle \overrightarrow{\sigma}_{y}\right\rangle \right)+\left(C_{0}^{0}\right)^{2}\Gamma\left\langle S^{x}\right\rangle ,\\
A_{k}^{yx} & =\Delta,\\
A_{k}^{yy} & =-\left(C_{0}^{0}\right)^{2}\Gamma\frac{\left(1+\left\langle \sigma_{z}^{k}\right\rangle \right)}{2},\\
A_{k}^{yz} & =2C_{0}^{0}\Omega\mathcal{R}\left({\rm e}^{{\rm i}\overrightarrow{{\rm k}}_{{\rm L}}\cdot\overrightarrow{r}_{k}}\left(\hat{e}_{{\rm L}}\cdot\hat{e}_{0}^{*}\right)\right)+\left(C_{0}^{0}\right)^{2}\left(\overrightarrow{\mathcal{R}}_{0,0}^{k}\cdot\left\langle \overrightarrow{\sigma}_{x}\right\rangle \right)+\left(C_{0}^{0}\right)^{2}\Gamma\left\langle S^{y}\right\rangle ,\\
A_{k}^{zx} & =-2C_{0}^{0}\Omega\mathcal{I}\left({\rm e}^{{\rm i}\overrightarrow{{\rm k}}_{{\rm L}}\cdot\overrightarrow{r}_{k}}\left(\hat{e}_{{\rm L}}\cdot\hat{e}_{0}^{*}\right)\right)+\left(C_{0}^{0}\right)^{2}\overrightarrow{\mathcal{R}}_{0,0}^{k}\cdot\left\langle \overrightarrow{\sigma}_{y}\right\rangle -\left(C_{0}^{0}\right)^{2}\Gamma\left(\left\langle S^{x}\right\rangle -\frac{\left\langle \sigma_{x}^{k}\right\rangle }{2}\right)\\
A_{k}^{zy} & =-2C_{0}^{0}\Omega\mathcal{R}\left({\rm e}^{{\rm i}\overrightarrow{{\rm k}}_{{\rm L}}\cdot\overrightarrow{r}_{k}}\left(\hat{e}_{{\rm L}}\cdot\hat{e}_{0}^{*}\right)\right)-\left(C_{0}^{0}\right)^{2}\overrightarrow{\mathcal{R}}_{0,0}^{k}\cdot\left\langle \overrightarrow{\sigma}_{x}\right\rangle -\left(C_{0}^{0}\right)^{2}\Gamma\left(\left\langle S^{y}\right\rangle -\frac{\left\langle \sigma_{y}^{k}\right\rangle }{2}\right)\\
A_{k}^{zz} & =-\left(C_{0}^{0}\right)^{2}\Gamma.
\end{align*}

We can observe that we can separate $A_{k}$ into an antisymmetric matrix and a “decay” matrix.

\begin{align*}
A_{k} & =C_{k}+D_{k},\\
C_{k}^{{\rm T}} & =-C_{k},\\
D_{k}^{xx} & =-\left(C_{0}^{0}\right)^{2}\Gamma\frac{\left(1+\left\langle \sigma_{z}^{k}\right\rangle \right)}{2},\\
D_{k}^{yy} & =-\left(C_{0}^{0}\right)^{2}\Gamma\frac{\left(1+\left\langle \sigma_{z}^{k}\right\rangle \right)}{2},\\
D_{k}^{zx} & =\left(C_{0}^{0}\right)^{2}\Gamma\frac{\left\langle \sigma_{x}^{k}\right\rangle }{2},\\
D_{k}^{zy} & =\left(C_{0}^{0}\right)^{2}\Gamma\frac{\left\langle \sigma_{y}^{k}\right\rangle }{2},\\
D_{k}^{zz} & =-\left(C_{0}^{0}\right)^{2}\Gamma,\\
C_{k}^{xy} & =-\Delta,\\
C_{k}^{xz} & =2C_{0}^{0}\Omega\mathcal{I}\left({\rm e}^{{\rm i}\overrightarrow{{\rm k}}_{{\rm L}}\cdot\overrightarrow{r}_{k}}\left(\hat{e}_{{\rm L}}\cdot\hat{e}_{0}^{*}\right)\right)-\left(C_{0}^{0}\right)^{2}\left(\overrightarrow{\mathcal{R}}_{0,0}^{k}\cdot\left\langle \overrightarrow{\sigma}_{y}\right\rangle \right)+\left(C_{0}^{0}\right)^{2}\Gamma\left\langle S^{x}\right\rangle ,\\
C_{k}^{yz} & =2C_{0}^{0}\Omega\mathcal{R}\left({\rm e}^{{\rm i}\overrightarrow{{\rm k}}_{{\rm L}}\cdot\overrightarrow{r}_{k}}\left(\hat{e}_{{\rm L}}\cdot\hat{e}_{0}^{*}\right)\right)+\left(C_{0}^{0}\right)^{2}\left(\overrightarrow{\mathcal{R}}_{0,0}^{k}\cdot\left\langle \overrightarrow{\sigma}_{x}\right\rangle \right)+\left(C_{0}^{0}\right)^{2}\Gamma\left\langle S^{y}\right\rangle .
\end{align*}

where:

\begin{align*}
\overrightarrow{\mathcal{R}}_{0,0}^{k} & =\begin{pmatrix}\mathcal{R}_{0,0}^{k0}\\
\mathcal{R}_{0,0}^{k1}\\
\vdots\\
\mathcal{R}_{0,0}^{k\left(N-1\right)}
\end{pmatrix},\\
\left\langle \overrightarrow{\sigma}_{\alpha}\right\rangle  & =\begin{pmatrix}\left\langle \sigma_{\alpha}^{0}\right\rangle \\
\left\langle \sigma_{\alpha}^{1}\right\rangle \\
\vdots\\
\left\langle \sigma_{\alpha}^{N-1}\right\rangle 
\end{pmatrix}.
\end{align*}

Writing in a more suitable form:


\begin{align*}
{\rm d}\left\langle \sigma_{x}^{k}\right\rangle  & =\left(-\Delta\left\langle \sigma_{y}^{k}\right\rangle +2C_{0}^{0}\Omega\left\langle \sigma_{z}^{k}\right\rangle \mathcal{I}\left({\rm e}^{{\rm i}\overrightarrow{{\rm k}}_{{\rm L}}\cdot\overrightarrow{r}_{k}}\left(\hat{e}_{{\rm L}}\cdot\hat{e}_{0}^{*}\right)\right)-\left(C_{0}^{0}\right)^{2}\left(\overrightarrow{\mathcal{R}}_{0,0}^{k}\cdot\left\langle \overrightarrow{\sigma}_{y}\right\rangle \right)\left\langle \sigma_{z}^{k}\right\rangle +\left(C_{0}^{0}\right)^{2}\Gamma\left(\left\langle S^{x}\right\rangle \left\langle \sigma_{z}^{k}\right\rangle -\frac{\left\langle \sigma_{x}^{k}\right\rangle \left(1+\left\langle \sigma_{z}^{k}\right\rangle \right)}{2}\right)\right){\rm d}t-C_{0}^{0}\sqrt{\Gamma}\left\langle \sigma_{y}^{k}\right\rangle {\rm d}W,\\
{\rm d}_{t}\left\langle \sigma_{y}^{k}\right\rangle  & =\left(\Delta\left\langle \sigma_{x}^{k}\right\rangle +2C_{0}^{0}\Omega\left\langle \sigma_{z}^{k}\right\rangle \mathcal{R}\left({\rm e}^{{\rm i}\overrightarrow{{\rm k}}_{{\rm L}}\cdot\overrightarrow{r}_{k}}\left(\hat{e}_{{\rm L}}\cdot\hat{e}_{0}^{*}\right)\right)+\left(C_{0}^{0}\right)^{2}\left\langle \sigma_{z}^{k}\right\rangle \left(\overrightarrow{\mathcal{R}}_{0,0}^{k}\cdot\left\langle \overrightarrow{\sigma}_{x}\right\rangle \right)+\left(C_{0}^{0}\right)^{2}\Gamma\left(\left\langle S^{y}\right\rangle \left\langle \sigma_{z}^{k}\right\rangle -\frac{\left\langle \sigma_{y}^{k}\right\rangle \left(1+\left\langle \sigma_{z}^{k}\right\rangle \right)}{2}\right)\right){\rm d}t+C_{0}^{0}\sqrt{\Gamma}\left\langle \sigma_{x}^{k}\right\rangle {\rm d}W,\\
{\rm d}_{t}\left\langle \sigma_{z}^{k}\right\rangle  & =\left(-2C_{0}^{0}\Omega\left(\left\langle \sigma_{x}^{k}\right\rangle \mathcal{I}\left({\rm e}^{{\rm i}\overrightarrow{{\rm k}}_{{\rm L}}\cdot\overrightarrow{r}_{k}}\left(\hat{e}_{{\rm L}}\cdot\hat{e}_{0}^{*}\right)\right)+\left\langle \sigma_{y}^{k}\right\rangle \mathcal{R}\left({\rm e}^{{\rm i}\overrightarrow{{\rm k}}_{{\rm L}}\cdot\overrightarrow{r}_{k}}\left(\hat{e}_{{\rm L}}\cdot\hat{e}_{0}^{*}\right)\right)\right)-\left(C_{0}^{0}\right)^{2}\overrightarrow{\mathcal{R}}_{0,0}^{k}\cdot\left(\left\langle \overrightarrow{\sigma}_{x}\right\rangle \left\langle \sigma_{y}^{k}\right\rangle -\left\langle \sigma_{x}^{k}\right\rangle \left\langle \overrightarrow{\sigma}_{y}\right\rangle \right)\right){\rm d}t\\
 & \,\,\,\,\,-\left(C_{0}^{0}\right)^{2}\Gamma\left(\left\langle \sigma_{x}^{k}\right\rangle \left\langle S^{x}\right\rangle +\left\langle \sigma_{y}^{k}\right\rangle \left\langle S^{y}\right\rangle +1+\left\langle \sigma_{z}^{k}\right\rangle -\frac{\left\langle \sigma_{x}^{k}\right\rangle ^{2}+\left\langle \sigma_{y}^{k}\right\rangle ^{2}}{2}\right){\rm d}t+C_{0}^{0}\sqrt{\Gamma}\left(1+\left\langle \sigma_{z}^{k}\right\rangle \right){\rm d}W.
\end{align*}

Define:

\begin{align*}
\mathcal{R}_{{\rm L}k} & \equiv\mathcal{R}\left({\rm e}^{{\rm i}\overrightarrow{{\rm k}}_{{\rm L}}\cdot\overrightarrow{r}_{k}}\left(\hat{e}_{{\rm L}}\cdot\hat{e}_{0}^{*}\right)\right),\\
\mathcal{I}_{{\rm L}k} & \equiv\mathcal{I}\left({\rm e}^{{\rm i}\overrightarrow{{\rm k}}_{{\rm L}}\cdot\overrightarrow{r}_{k}}\left(\hat{e}_{{\rm L}}\cdot\hat{e}_{0}^{*}\right)\right).
\end{align*}

In this case:


\begin{align*}
{\rm d}\left\langle \sigma_{x}^{k}\right\rangle  & =\left(\!-\Delta\left\langle \sigma_{y}^{k}\right\rangle +2C_{0}^{0}\Omega\left\langle \sigma_{z}^{k}\right\rangle \mathcal{I}_{{\rm L}k}-\left(C_{0}^{0}\right)^{4}\overrightarrow{\mathcal{R}}_{0,0}^{k}\cdot\left\langle \overrightarrow{\sigma}_{y}\right\rangle \!\left\langle \sigma_{z}^{k}\right\rangle +\left(C_{0}^{0}\right)^{2}\Gamma\left(\!\left\langle S^{x}\right\rangle \left\langle \sigma_{z}^{k}\right\rangle -\frac{\left\langle \sigma_{x}^{k}\right\rangle \left(1+\left\langle \sigma_{z}^{k}\right\rangle \right)}{2}\!\right)\!\right){\rm d}t-C_{0}^{0}\sqrt{\Gamma}\left\langle \sigma_{y}^{k}\right\rangle {\rm d}W,\\
{\rm d}_{t}\left\langle \sigma_{y}^{k}\right\rangle  & =\left(\!\Delta\left\langle \sigma_{x}^{k}\right\rangle +2C_{0}^{0}\Omega\left\langle \sigma_{z}^{k}\right\rangle \mathcal{R}_{{\rm L}k}+\left(C_{0}^{0}\right)^{4}\left\langle \sigma_{z}^{k}\right\rangle \left(\overrightarrow{\mathcal{R}}_{0,0}^{k}\cdot\left\langle \overrightarrow{\sigma}_{x}\right\rangle \right)+\left(C_{0}^{0}\right)^{2}\Gamma\left(\!\left\langle S^{y}\right\rangle \left\langle \sigma_{z}^{k}\right\rangle -\frac{\left\langle \sigma_{y}^{k}\right\rangle \left(1+\left\langle \sigma_{z}^{k}\right\rangle \right)}{2}\!\right)\!\right){\rm d}t+C_{0}^{0}\sqrt{\Gamma}\left\langle \sigma_{x}^{k}\right\rangle {\rm d}W,\\
{\rm d}_{t}\left\langle \sigma_{z}^{k}\right\rangle  & =\left(-2C_{0}^{0}\Omega\left(\left\langle \sigma_{x}^{k}\right\rangle \mathcal{I}_{{\rm L}k}+\left\langle \sigma_{y}^{k}\right\rangle \mathcal{R}_{{\rm L}k}\right)-\left(C_{0}^{0}\right)^{4}\overrightarrow{\mathcal{R}}_{0,0}^{k}\cdot\left(\left\langle \overrightarrow{\sigma}_{x}\right\rangle \left\langle \sigma_{y}^{k}\right\rangle -\left\langle \sigma_{x}^{k}\right\rangle \left\langle \overrightarrow{\sigma}_{y}\right\rangle \right)\right){\rm d}t+C_{0}^{0}\sqrt{\Gamma}\left(1+\left\langle \sigma_{z}^{k}\right\rangle \right){\rm d}W\\
 & \,\,\,\,\,-\left(C_{0}^{0}\right)^{2}\Gamma\left(\left\langle \sigma_{x}^{k}\right\rangle \left\langle S^{x}\right\rangle +\left\langle \sigma_{y}^{k}\right\rangle \left\langle S^{y}\right\rangle +1+\left\langle \sigma_{z}^{k}\right\rangle -\frac{\left\langle \sigma_{x}^{k}\right\rangle ^{2}+\left\langle \sigma_{y}^{k}\right\rangle ^{2}}{2}\right){\rm d}t.
\end{align*}

In order to perform the sampling using the rotated basis consider the following equation:


\begin{equation}
{\rm d}\left\langle \overrightarrow{S}^{{\rm R}}\right\rangle ={\rm R\left(\left(A\left({\rm R}^{{\rm T}}\left\langle \overrightarrow{S}^{{\rm R}}\right\rangle \right){\rm R}^{{\rm T}}\left\langle \overrightarrow{S}^{{\rm R}}\right\rangle +\overrightarrow{b}\left({\rm R}^{{\rm T}}\left\langle \overrightarrow{S}^{{\rm R}}\right\rangle \right)\right){\rm d}t+\overrightarrow{g}\left({\rm R}^{{\rm T}}\left\langle \overrightarrow{S}^{{\rm R}}\right\rangle \right){\rm d}W\right)}.
\end{equation}

In [4]:
t0=time.time()

# Parameters

In [5]:
#Parameters  (Check the units)

Gamma_r=1.44#  2*pi*6e6  #reference

wL=4.5*Gamma_r  #Laser frequency
w0=4.5*Gamma_r  #Energy difference, twith hbar=1
c_light=3e8     #m/s



DeltaL0=wL-w0      #wL − w0, where wL is the laser detuning   , What are the units?
Omega=4.5*Gamma_r  #Driving frequency , What are the units?
N_atom=4           #Number of atoms

lambda_r=2    # in m

Lambda_las=1     # in m    Gamma_r*2*pi/(wL/c_light)   #Laser wavelenght  , What are the units?

e_L=array([0,1,1j]/sqrt(2))


Theta1=pi/2
Theta2=pi/2   #they got [0,1,0]

e_0=array([sin(Theta1)*cos(Theta2),sin(Theta1)*sin(Theta2),cos(Theta1)])

K_laser=(2*pi/Lambda_las)*array([1,0,0])

k0=2*pi/Lambda_las

Gamma_dec=Gamma_r

# Clebsch-Gordan coefficient

In [6]:
# Clebsch-Gordan coefficient

C00=1    #will change due to the transition nature
   



# Positions

In [7]:
# Positions


## Oblate Variances matrix: Set the shape of the atoms cloud

Sigma_x=20 *lambda_r    
Sigma_y=0.6*lambda_r    
Sigma_z=Sigma_y        


Oblate_variances=zeros((3,3))

Oblate_variances[0][0]=Sigma_x
Oblate_variances[1][1]=Sigma_y
Oblate_variances[2][2]=Sigma_z

Miu_pos=[0 , 0, 0]

Pos_atoms=(random.multivariate_normal(Miu_pos, Oblate_variances, N_atom))



# Construct $I_{Lk}$ and $R_{Lk}$

In [8]:
# Construct I_Lk and R_Lk


eL_e0c=dot(e_L,conj(e_0))


DotKL_rk=[]   #summarize k_L.r_k for all k

for j in range(N_atom):
    DotKL_rk.append(dot(Pos_atoms[j],K_laser))

DotKL_rk_exp=exp(1j*array(DotKL_rk))


ImL_k=imag(DotKL_rk_exp*eL_e0c)
RmL_k=real(DotKL_rk_exp*eL_e0c)


# Construct $R_{00}$

In [9]:
# Construct the matrix R00
t0=time.time()


def fun_Pkl0(k,l,k0=k0):
    
    VecR_kl=Pos_atoms[k] -Pos_atoms[l]
    R_kl=sqrt(dot(VecR_kl,VecR_kl))
    #R_kl=norm(VecR_kl)
    
    
    Pkl0=dot(conj(e_0),VecR_kl)
    Pkl_0nC=(abs(Pkl0))**2
    
    M0kl=k0*R_kl
    
    return (1-Pkl_0nC)/M0kl+(1-3*Pkl_0nC)*(1j/(M0kl)**2-1/(M0kl)**3)
    


MatL=zeros((N_atom,N_atom),dtype='complex')
MatR=zeros((N_atom,N_atom),dtype='complex')


for i in range(N_atom):
    for j in range(i+1,N_atom):
 
        
        MatL[i][j]=exp(1j*k0*dot(e_0,(Pos_atoms[i] -Pos_atoms[j])))
        MatR[i][j]=fun_Pkl0(i,j,k0)

        
        
MatR=MatR+MatR.T
MatL=MatL+conj(MatL.T)

R00=real(MatL*MatR)

t1=time.time()

In [10]:
print(t1-t0)

0.0010023117065429688


In [11]:
N_atom

4

\begin{align*}
A_{k} & =C_{k}+D_{k},\\
C_{k}^{{\rm T}} & =-C_{k},\\
D_{k}^{xx} & =-\left(C_{0}^{0}\right)^{2}\Gamma\frac{\left(1+\left\langle \sigma_{z}^{k}\right\rangle \right)}{2},\\
D_{k}^{yy} & =-\left(C_{0}^{0}\right)^{2}\Gamma\frac{\left(1+\left\langle \sigma_{z}^{k}\right\rangle \right)}{2},\\
D_{k}^{zx} & =\left(C_{0}^{0}\right)^{2}\Gamma\frac{\left\langle \sigma_{x}^{k}\right\rangle }{2},\\
D_{k}^{zy} & =\left(C_{0}^{0}\right)^{2}\Gamma\frac{\left\langle \sigma_{y}^{k}\right\rangle }{2},\\
D_{k}^{zz} & =-\left(C_{0}^{0}\right)^{2}\Gamma,\\
C_{k}^{xy} & =-\Delta,\\
C_{k}^{xz} & =2C_{0}^{0}\Omega\mathcal{I}\left({\rm e}^{{\rm i}\overrightarrow{{\rm k}}_{{\rm L}}\cdot\overrightarrow{r}_{k}}\left(\hat{e}_{{\rm L}}\cdot\hat{e}_{+}^{*}\right)\right)-\left(C_{0}^{0}\right)^{4}\left(\overrightarrow{\mathcal{R}}_{0,0}^{k}\cdot\left\langle \overrightarrow{\sigma}_{y}\right\rangle \right)+\left(C_{0}^{0}\right)^{2}\Gamma\left\langle S^{x}\right\rangle ,\\
C_{k}^{yz} & =2C_{0}^{0}\Omega\mathcal{R}\left({\rm e}^{{\rm i}\overrightarrow{{\rm k}}_{{\rm L}}\cdot\overrightarrow{r}_{k}}\left(\hat{e}_{{\rm L}}\cdot\hat{e}_{+}^{*}\right)\right)+\left(C_{0}^{0}\right)^{4}\left(\overrightarrow{\mathcal{R}}_{0,0}^{k}\cdot\left\langle \overrightarrow{\sigma}_{x}\right\rangle \right)+\left(C_{0}^{0}\right)^{2}\Gamma\left\langle S^{y}\right\rangle .
\end{align*}


\begin{equation}
{\rm d}\left\langle \overrightarrow{s}_{k}\right\rangle =A_{k}\left\langle \overrightarrow{s}_{k}\right\rangle {\rm d}t-\left(C_{0}^{0}\right)^{2}\Gamma\begin{pmatrix}0\\
0\\
1
\end{pmatrix}{\rm d}t+C_{0}^{0}\sqrt{\Gamma}\begin{pmatrix}-\left\langle \sigma_{y}^{k}\right\rangle \\
\left\langle \sigma_{x}^{k}\right\rangle \\
1+\left\langle \sigma_{z}^{k}\right\rangle 
\end{pmatrix}{\rm d}W
\end{equation}

# Construct matrix $A_k$

In [12]:

def Matrix_Ak(Pk,VectSx,VectSy,VectSz,DeltaL0=DeltaL0,C00=C00,Omega=Omega,Gamma_dec=Gamma_dec,N_atom=N_atom):
    
    """
    Pk: Index of the kth atom
    VectSy: \overrightarrow{\sigma_y}
    VectSx: \overrightarrow{\sigma_X}
    """
    
    Cxy_k=-DeltaL0
    Cxz_k=2*C00*Omega*ImL_k[Pk]-(C00**4)*dot(R00[k],VectSy)+(C00**2)*Gamma_dec*sum(VectSx)/2
    Cyz_k=2*C00*Omega*RmL_k[Pk]-(C00**4)*dot(R00[k],VectSy)+(C00**2)*Gamma_dec*sum(VectSy)/2
    
    Transl_i=3*Pk
    
    I = array([0,0,1])+Transl_i
    J = array([1,2,2])+Transl_i
    V = array([Cxy_k,Cxz_k,Cyz_k])
    
    Gen_k = sparse.coo_matrix((V,(I,J)),shape=(3*N_atom,3*N_atom))
    
    C_k=Gen_k- Gen_k.T
    
    sxk=VectSx[Pk]
    syk=VectSy[Pk]
    szk=VectSz[Pk]
    
    
    refz=(1+szk)/2
    
    Dxx_k=-(C00**2)*Gamma_dec*refz
    Dyy_k=-(C00**2)*Gamma_dec*refz
    Dzx_k= (C00**2)*Gamma_dec*sxk/2
    Dzy_k= (C00**2)*Gamma_dec*syk/2
    Dzz_k=-(C00**2)*Gamma_dec
    
    
    I = array([0,1,2,2,2])+Transl_i
    J = array([0,1,0,1,2])+Transl_i
    V = array([Dxx_k,Dyy_k,Dzx_k,Dzy_k,Dzz_k])
    
    D_k = sparse.coo_matrix((V,(I,J)),shape=(3*N_atom,3*N_atom))
    
    
    
    return C_k+D_k


# Construct matrix $A$

In [13]:
def Matrix_A(VectSy,VectSx,VectSz,DeltaL0=DeltaL0,C00=C00,Omega=Omega,Gamma_dec=Gamma_dec,N_atom=N_atom):
    
    Mat_A=Matrix_Ak(0,VectSy,VectSx,VectSz,DeltaL0=DeltaL0,C00=C00,Omega=Omega,Gamma_dec=Gamma_dec,N_atom=N_atom)
    
    for j in range(1,N_atom):
        Mat_A=Mat_A+Matrix_Ak(j,VectSy,VectSx,VectSz,DeltaL0=DeltaL0,C00=C00,Omega=Omega,Gamma_dec=Gamma_dec,N_atom=N_atom)
        
    return Mat_A

# State Parameters

In [14]:
# State parameters


Alpha=0

# The angle is associated with the position via Klaser.rk



# Local rotation matrix

In [15]:
def MatrixRot_k(Pk,K_laser=K_laser,Alpha=Alpha):
    
    Rk=Pos_atoms[Pk]
    
    Phi_k=dot(K_laser,Rk)
    
    S_k=sin(Phi_k)
    C_k=cos(Phi_k)
    S_a=sin(2*Alpha)
    C_a=cos(2*Alpha)
    
    Array_Rot_k=np.array([[-C_k*C_a, S_k*C_a,-S_a],
                        [S_k,C_k,0],
                        [C_k*S_a,-S_k*S_a,-C_a]])
    
    return Array_Rot_k.round(6)

In [16]:
def gRot_gen():
    Array_Rot_gen=np.array([[0,-1,0],
                           [1,0,0],
                           [0,0,1]])
    
    return Array_Rot_gen

    

In [17]:
# Matrix R of general rotation

tu0=time.time()

round_val=6
k=0
M_k=csr_matrix(MatrixRot_k(0).round(round_val))

Coord=(M_k).nonzero()

Xk=Coord[0]
Yk=Coord[1]
Valk=(M_k).data

X_arr=Xk
Y_arr=Yk
Val_arr=Valk


for k in range(1,N_atom):
    M_k=csr_matrix(MatrixRot_k(k).round(round_val))
    Coord=(M_k).nonzero()
    
    Xk=Coord[0]+3*k
    Yk=Coord[1]+3*k
    Valk=(M_k).data
    
    X_arr=np.concatenate((X_arr, Xk), axis=0)
    Y_arr=np.concatenate((Y_arr, Yk), axis=0)
    Val_arr=np.concatenate((Val_arr, Valk), axis=0)
    
    
Rot_R = sparse.coo_matrix((Val_arr,(X_arr,Y_arr)),shape=(3*N_atom,3*N_atom))  

tu1=time.time()

print(tu1-tu0)

0.003999948501586914


In [18]:
Val_arr=np.array(N_atom*[-1,1,1])

Ib=[]
Jb=[]

for j in range(N_atom):
    
    Ij=np.array([0+3*j,1+3*j,2+3*j])
    Jj=np.array([1+3*j,0+3*j,2+3*j])
    
    Ib= np.concatenate((Ib, Ij))
    Jb= np.concatenate((Jb, Jj))


In [19]:
Rot_Val = sparse.coo_matrix((array(Val_arr),(Ib,Jb)),shape=(3*N_atom,3*N_atom))

In [20]:
#print(Rot_R)

In [21]:
sqr_GammaR=sqrt(Gamma_dec)

In [22]:
#Plot the matrix R


#import numpy as np
#import matplotlib.pyplot as plt

#plt.imshow(Rot_R.todense())
#plt.colorbar()
#plt.show()

In [23]:
#More efficient multiplication


#tyu0=time.time()

#Vali=Rot_R@rty

#tyu1=time.time()

# Sampling

In [24]:

def init_state(N_atom=N_atom):
    elements = [-1, 1]
    probabilities = [0.5, 0.5]
    Measurements=list(choice(elements, 2*N_atom, p=probabilities))
    for j in range(N_atom):

        Measurements.insert(2+3*j,1)
        
    return Measurements
    


In [25]:
# Separate in Sx, Sy and Sz
@jit
def get_s_alpha(Measur):
    
    Arr_retX=[]
    Arr_retY=[]
    Arr_retZ=[]
    
    for j in range(N_atom):
        Arr_retX.append(Measur[3*j])
        Arr_retY.append(Measur[3*j+1])
        Arr_retZ.append(Measur[3*j+2])
    
    return Arr_retX,Arr_retY,Arr_retZ

In [26]:

def b_fun(N_atom=N_atom,Gamma_dec=Gamma_dec):
    
    b_column=[]
    
    for j in range(N_atom):
        b_column.append(0)
        b_column.append(0)
        b_column.append(1)
        
    return np.array(b_column)

In [27]:
TransR_m=Rot_R.T


b_funct=b_fun().reshape(3*N_atom,)



In [28]:
#%% Differential equation RHS function

#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

@jit
def f(yR, t):
    
    
    # Here yR is an array in the R xR,yR and zR basis. 
    
    
    # yOr is the results array in the x, y and z basis.
    
    
    yOr=(TransR_m)@yR
    
    Separ_alpha=get_s_alpha(yOr)
    
    
    yOr_x=Separ_alpha[0] 
    yOr_y=Separ_alpha[1] 
    yOr_z=Separ_alpha[2]  
    
    MatA=Rot_R@((Matrix_A(yOr_y,yOr_x,yOr_z)@TransR_m)@yR-(C00**2)*Gamma_dec*b_funct)


    return array(MatA)
    



In [29]:

#%% Noise function

@jit
def g(yR, t):
    
    
    # Here yR is an array in the R xR,yR and zR basis. 
    # yOr is the results array in the x, y and z basis.
    
    

    yOr=(TransR_m)@((array(yR)).reshape(3*N_atom,))
    
    return Rot_R@((C00)*sqr_GammaR*(    Rot_Val@yOr +b_funct))
    

# Euler-Maruyama

In [30]:
tspan=np.linspace(0,0.2,800+1)


h=(tspan[-1]-tspan[0])/(len(tspan)-1)


from numpy import zeros

In [31]:



@jit
def EulerMaruyama(f, g, y0 , tspan ):
    
    
    Stochastic=1
    
    y0,Ns =array(y0).reshape(3*N_atom,), len(tspan)

    h,y = (tspan[Ns-1] - tspan[0])/(Ns - 1),zeros((Ns, 3*N_atom), dtype=float)
    # allocate space for result

    y[0] = y0.reshape(3*N_atom,);
    
    Second_moment=[]
    
    sqrth=sqrt(h)*Stochastic
    
    init0=y0.reshape(3*N_atom,1)
    
    Second_moment.append( init0@(init0.T)   )
    
    
    for n in range(0, Ns-1):


        tn   = tspan[n]
        tnp1 = tspan[n+1]
        yn   = y[n]
        


        fn = f(yn, tn)
        gn = g(yn, tn)
        
        ynp1=yn + fn*h + gn*sqrth*normal()
        
        y[n+1] = ynp1
        
        Vect_np1=ynp1.reshape(3*N_atom,1)
        
        Second_moment.append( Vect_np1@(Vect_np1.T)   )
        
        #print(n)
        
    return y,Second_moment

In [32]:
yR=init_state()


y0=array(yR).reshape(3*N_atom,)


%timeit EulerMaruyama(f,g,yR,tspan)

<ipython-input-31-5595ec40d521>:1: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "EulerMaruyama" failed type inference due to: No implementation of function Function(<built-in function zeros>) found for signature:
 
 >>> zeros(UniTuple(int64 x 2), dtype=Function(<class 'float'>))
 
There are 2 candidate implementations:
  - Of which 2 did not match due to:
  Overload of function 'zeros': File: numba\core\typing\npydecl.py: Line 526.
    With argument(s): '(UniTuple(int64 x 2), dtype=Function(<class 'float'>))':
   No match.

During: resolving callee type: Function(<built-in function zeros>)
During: typing of call at <ipython-input-31-5595ec40d521> (9)


File "<ipython-input-31-5595ec40d521>", line 9:
def EulerMaruyama(f, g, y0 , tspan ):
    <source elided>

    h,y = (tspan[Ns-1] - tspan[0])/(Ns - 1),zeros((Ns, 3*N_atom), dtype=float)
    ^

  @jit
<ipython-input-31-5595ec40d521>:1: NumbaWarning: 
Compilation is falling back to obj

1.78 s ± 40.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [33]:
res1=EulerMaruyama(f,g,yR,tspan)

In [34]:
@jit

def SDE_sol():

    return EulerMaruyama(f, g, init_state(), tspan)

In [35]:
%timeit SDE_sol()

<ipython-input-34-1aafa4f67cef>:1: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "SDE_sol" failed type inference due to: Untyped global name 'init_state': cannot determine Numba type of <class 'function'>

File "<ipython-input-34-1aafa4f67cef>", line 5:
def SDE_sol():
    <source elided>

    return EulerMaruyama(f, g, init_state(), tspan)
    ^

  @jit
C:\Users\camil\anaconda3\lib\site-packages\numba\core\object_mode_passes.py:177: NumbaWarning: Function "SDE_sol" was compiled in object mode without forceobj=True.

File "<ipython-input-34-1aafa4f67cef>", line 3:

def SDE_sol():
^

  warnings.warn(errors.NumbaWarning(warn_msg,
C:\Users\camil\anaconda3\lib\site-packages\numba\core\object_mode_passes.py:187: NumbaDeprecationWarning: 
Fall-back from the nopython compilation path to the object mode compilation path has been detected, this is deprecated behaviour.

For more information visit http://numba.pydata.org/numba-doc/latest/refer

1.81 s ± 41.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [36]:





itera=500
@jit

def averages_sde(itera=itera,N_atom=N_atom,tspan=tspan):
    Results =EulerMaruyama(f,g,init_state(),tspan)
    
    Result_First  =Results[0]
    Second_moment =Results[1]

    print(0)

    for j in range(itera-1):
        Trayec=EulerMaruyama(f,g,init_state(),tspan)
        Result_First  =add(Result_First  ,Trayec[0])
        Second_moment =add(Second_moment, Trayec[1])

        print(j+1)
    
    return np.array(Result_First)/int(itera), np.array(Second_moment)/int(itera)

In [37]:
import time
t0=time.time()

itera=60
Result_varTWA_Col_SDE = averages_sde(itera=itera,N_atom=N_atom,tspan=tspan)
#Result_varTWA_Col_TWA = averages_sde(itera=itera,N=N)

t1=time.time()

print(t1-t0)

<ipython-input-36-77d0062f5b40>:2: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "averages_sde" failed type inference due to: Untyped global name 'init_state': cannot determine Numba type of <class 'function'>

File "<ipython-input-36-77d0062f5b40>", line 5:
def averages_sde(itera=itera,N_atom=N_atom,tspan=tspan):
    Results =EulerMaruyama(f,g,init_state(),tspan)
    ^

  @jit
<ipython-input-36-77d0062f5b40>:2: NumbaWarning: 
Compilation is falling back to object mode WITHOUT looplifting enabled because Function "averages_sde" failed type inference due to: Untyped global name 'init_state': cannot determine Numba type of <class 'function'>

File "<ipython-input-36-77d0062f5b40>", line 5:
def averages_sde(itera=itera,N_atom=N_atom,tspan=tspan):
    Results =EulerMaruyama(f,g,init_state(),tspan)
    ^

  @jit
C:\Users\camil\anaconda3\lib\site-packages\numba\core\object_mode_passes.py:177: NumbaWarning: Function "averages_sde" was comp

0


<ipython-input-36-77d0062f5b40>:2: NumbaWarning: 
Compilation is falling back to object mode WITHOUT looplifting enabled because Function "averages_sde" failed type inference due to: Untyped global name 'init_state': cannot determine Numba type of <class 'function'>

File "<ipython-input-36-77d0062f5b40>", line 13:
def averages_sde(itera=itera,N_atom=N_atom,tspan=tspan):
    <source elided>
    for j in range(itera-1):
        Trayec=EulerMaruyama(f,g,init_state(),tspan)
        ^

  @jit
C:\Users\camil\anaconda3\lib\site-packages\numba\core\object_mode_passes.py:177: NumbaWarning: Function "averages_sde" was compiled in object mode without forceobj=True.

File "<ipython-input-36-77d0062f5b40>", line 12:
def averages_sde(itera=itera,N_atom=N_atom,tspan=tspan):
    <source elided>

    for j in range(itera-1):
    ^

  warnings.warn(errors.NumbaWarning(warn_msg,
C:\Users\camil\anaconda3\lib\site-packages\numba\core\object_mode_passes.py:187: NumbaDeprecationWarning: 
Fall-back from the 

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
130.6959331035614


In [49]:
@jit

def SDE_sol():
    
    Sum_TWA=np.random.multivariate_normal(N*real(Miu), Cov_mat_list_alt, 1).T
    
    y0=list(Sum_TWA.flatten())+[random.normal(0, 0.5),random.normal(0, 0.5),random.normal(0, 0.5),random.normal(0, 0.5)]
    
    return EulerMaruyama(f, GG, y0, t_alt)

In [50]:
y0=init_state()

In [51]:
y0=array(y0).reshape(3*N_atom,1)

In [52]:
Ns = len(tspan)
h = (tspan[Ns-1] - tspan[0])/(Ns - 1)
    # allocate space for result
y = np.zeros((Ns, 3*N_atom), dtype=type(y0[0]))

In [53]:
#@jit
def hui(arr):
    
    return arr.reshape(3*N_atom,)

In [54]:
#(np.array(init_state()).reshape(3*N_atom,))

hui(y0)


array([ 1,  1,  1,  1,  1,  1,  1,  1,  1, -1,  1,  1])

In [55]:

th1=time.time()


y0=np.array(init_state())

num=300000

for j in range(num):
    
    #y0.reshape(3*N_atom,)
    
    
    hui(y0)

th2=time.time()

print((th2-th1)/num,th2-th1)


1.0437917709350586e-06 0.3131375312805176


In [56]:
    

    Ns = len(tspan)
    h = (tspan[Ns-1] - tspan[0])/(Ns - 1)
    # allocate space for result
    y = np.zeros((Ns, 3*N_atom), dtype=type(y0[0]))

    y[0] = y0.reshape(3*N_atom,);
    for n in range(0, Ns-1):


        tn = tspan[n]
        tnp1 = tspan[n+1]
        yn = y[n]
        


        fn = f(yn, tn)
        gn = g(yn, tn)
        
        y[n+1] = yn + fn*h  + gn*sqrt(h)*normal()
        

<ipython-input-28-863c90265cca>:5: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "f" failed type inference due to: Untyped global name 'TransR_m': cannot determine Numba type of <class 'scipy.sparse.coo.coo_matrix'>

File "<ipython-input-28-863c90265cca>", line 15:
def f(yR, t):
    <source elided>
    
    yOr=(TransR_m)@yR
    ^

  @jit
C:\Users\camil\anaconda3\lib\site-packages\numba\core\object_mode_passes.py:177: NumbaWarning: Function "f" was compiled in object mode without forceobj=True.

File "<ipython-input-28-863c90265cca>", line 6:
@jit
def f(yR, t):
^

  warnings.warn(errors.NumbaWarning(warn_msg,
C:\Users\camil\anaconda3\lib\site-packages\numba\core\object_mode_passes.py:187: NumbaDeprecationWarning: 
Fall-back from the nopython compilation path to the object mode compilation path has been detected, this is deprecated behaviour.

For more information visit http://numba.pydata.org/numba-doc/latest/reference/deprecation.h